In [1]:
# Modelo de prediccion 

import sys
module_path = "../../src"
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import numpy as np
import cv2
import os

    
from detector.detector import detector
from modelo_firmas.inference import firmas_model


PATH_DATASET = '../../01.Datasets/'

PATH_IMG_DOCUMENT = os.path.join(PATH_DATASET,"files/reference.png")
PATH_RESULTS_MODEL = '../../03.SavedModels/modelo_firmas.h5'   

2021-08-21 17:52:07.953199: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
my_detector = detector(PATH_IMG_DOCUMENT)

In [3]:
my_firmas_model = firmas_model(PATH_RESULTS_MODEL)

2021-08-21 17:52:08.968755: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-08-21 17:52:09.006302: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-21 17:52:09.006835: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2070 computeCapability: 7.5
coreClock: 1.455GHz coreCount: 36 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 327.88GiB/s
2021-08-21 17:52:09.006899: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-08-21 17:52:09.009500: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-08-21 17:52:09.009528: I tensorflow/stream_executor/pla

In [4]:
df_train = pd.read_csv(os.path.join(PATH_DATASET,'files/output_train.csv'),dtype=str)
df_test = pd.read_csv(os.path.join(PATH_DATASET,'files/sampleSubmission.csv'),dtype=str)

In [5]:
def get_fecha(img):
    #Codigo de OCR#
    
    return [0,0,2021]

In [6]:
firma1_list = []
firma2_list = []

day_list = []
month_list = []
year_list = []

for idx in tqdm(range(len(df_test))):
    id_name = df_test.iloc[idx]['id']

    imFilename = os.path.join(PATH_DATASET,'images_test',id_name)+'.jpg'
    img = cv2.imread(imFilename)
    img_aligned,fecha,firma1,firma2= my_detector.predict(img)
    firma1_class,score1 = my_firmas_model.predict(firma1)
    firma2_class,score2 = my_firmas_model.predict(firma2)
    
    [day,month,year] = get_fecha(fecha)
    
    firma1_list.append(firma1_class)
    firma2_list.append(firma2_class)

    day_list.append(day)
    month_list.append(month)
    year_list.append(year)


  0%|          | 0/107 [00:00<?, ?it/s]

2021-08-21 17:52:11.748995: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-08-21 17:52:11.749554: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2599990000 Hz
2021-08-21 17:52:12.762165: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2021-08-21 17:52:13.077260: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8200
2021-08-21 17:52:13.442209: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-08-21 17:52:13.811918: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


In [7]:
df_test.columns

Index(['id', 'sign_1', 'sign_2', 'date_day', 'date_month', 'date_year'], dtype='object')

In [8]:
df_test['sign_1']= firma1_list
df_test['sign_2']= firma2_list
df_test['date_day']= day_list
df_test['date_month']= month_list
df_test['date_year']= year_list


In [9]:
df_test['n'] = df_test['id'].str.split('_').str[-1]
df_test['n'] = pd.to_numeric(df_test['n'], errors='coerce').fillna(0).astype(np.int64)

df_test = df_test.sort_values('n')
del df_test['n']

df_test.to_csv('submissions/submission_.csv',index=0)

In [10]:
df_test.head()

,id,sign_1,sign_2,date_day,date_month,date_year
6,C_2,1,1,0,0,2021
47,C_4,1,1,0,0,2021
81,C_9,1,1,0,0,2021
31,C_11,1,1,0,0,2021
56,C_16,1,1,0,0,2021
